In [8]:
import os
import json
import pandas as pd
import traceback
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv


In [10]:
load_dotenv()
KEY = os.getenv("OPENAI-API-KEY")

In [6]:
llm = ChatOpenAI(open_ai_apikey = KEY, model_name = "gpt-3.5-turbo", temperature = 0.5)

WARNING! open_ai_apikey is not default parameter.
                    open_ai_apikey was transferred to model_kwargs.
                    Please confirm that open_ai_apikey is what you intended.


ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)